In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob as gl
import os

In [2]:
path = "src/results"

""" datetime = "/2011-03-24_15"
df = pd.read_csv(path + datetime + '.csv')  """


dfs = []

for file in gl.glob(path + '/20*.csv'):
    dfs.append(pd.read_csv(file))

df = pd.concat(dfs, ignore_index=True)
#df.count() #cada csv tiene 160920 filas

## Encoder

In [3]:
modelo = df.iloc[:, 8].values 
from sklearn.preprocessing import LabelEncoder
modelo = LabelEncoder().fit_transform(modelo) #codificación del modelo

In [4]:
df = df.drop(df.columns[8], axis=1) #se elimina la antigua con los strings del modelo
df['modelo'] = modelo #se añade la nueva codificada al final

X = df.iloc[:, 1:] 
X = X.drop(['datetime', 'timestamp', 'load', 'DC Array Output (W)' , 'Pavg', 'dif'], axis=1)
y = df.iloc[:, 0].values #valores de overflow

X.columns

Index(['cap', 'dist', 'origen_id', 'dest_id', 'len_origen_tag', 'len_dest_tag',
       'criterion', 'degree', 'total_balance', 'abs_flux', 'h',
       'Beam Irradiance (W/m2)', 'Diffuse Irradiance (W/m2)',
       'Ambient Temperature (C)', 'Plane of Array Irradiance (W/m2)',
       'Cell Temperature (C)', 'modelo'],
      dtype='object')

## SVM

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() #escalado de datos
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from sklearn.svm import SVC #ESTO HA TARDADO LA VIDA Y MEDIA....
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

## Evaluation

### Confussion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[94436   249]
 [ 1461   406]]


0.982289336316182

### K-Fold

In [ ]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

' from sklearn.model_selection import cross_val_score\n\naccuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10) #se añade el clasificador o modelo, cv = K\nprint("Accuracy: {:.2f} %".format(accuracies.mean()*100))\nprint("Standard Deviation: {:.2f} %".format(accuracies.std()*100)) '

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

' from sklearn.model_selection import GridSearchCV\nfrom sklearn.neural_network import MLPClassifier\n\nparameters = {\n    \'hidden_layer_sizes\': [(5,5,1), (6,6,1), (7,7,1), (8,8,1)],\n    \'activation\': [\'logistic\', \'tanh\',\'relu\'],\n    \'solver\': [\'lbfgs\', \'sgd\',\'adam\'],\n    #\'batch_size\': [25, 50, 75, 100],\n    #\'epochs\': [10, 20, 30],\n    \'learning_rate\': [\'constant\',\'adaptive\'],\n}\n\ngrid_search = GridSearchCV(MLPClassifier(max_iter=100), parameters, n_jobs=-1, cv=10, scoring = \'accuracy\')\ngrid_search.fit(X_train, y_train)\n\nbest_accuracy = grid_search.best_score_\nbest_parameters = grid_search.best_params_\nprint("Best Accuracy: {:.2f} %".format(best_accuracy*100))\nprint("Best Parameters:", best_parameters)\n\n\n########\nBest Accuracy: 98.20 %\nBest Parameters: {\'activation\': \'tanh\', \'hidden_layer_sizes\': (7, 7, 1), \'learning_rate\': \'constant\', \'solver\': \'adam\'} '